In [14]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# initialize weights and bias

def initialize_parameters(layer_dims):

    params = {}  # initialize a dictionary for storing parameters
    L = len(layer_dims)  # number of layers in the network

    for l in range(1, L):
        params['w'+str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01 # initialize weights
        params['b'+str(l)] = np.zeros((layer_dims[l], 1))   # initialize bias

        assert(params['w'+str(l)].shape == (layer_dims[l], layer_dims[l-1])) # check the shape of weights
        assert(params['b'+str(l)].shape == (layer_dims[l], 1)) # check the shape of bias

    return params

In [4]:
# Forward propagation

def forward_pg (A, W, b):
    Z = np.dot(W, A) + b # compute the linear combination

    assert(Z.shape == (W.shape[0], A.shape[1])) # check the shape of linear combination
    cache = (A, W, b) # store the linear combination in cache

    return Z, cache

In [5]:
# activation function

def activation(A_prev, W, b, activation_type):

    if activation_type == "sigmoid":
        Z, linear_cache = forward_pg(A_prev, W, b) # compute linear combination
        A, activation_cache = sigmoid(Z) # compute the sigmoid

    elif activation_type == "relu":
        Z, linear_cache = forward_pg(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    else:
        print("Wrong activation type!")
        print("Please choose from sigmoid or relu!")
        exit(1)
    
    assert(A.shape == (W.shape[0], A_prev.shape[1])) # check the shape of activation
    cache = (linear_cache, activation_cache) # store the linear and activation combination in cache

    return A, cache


def sigmoid(Z):
        A = 1/(1+np.exp(-Z)) # compute the sigmoid
        cache = Z

        return A, cache

def relu(Z):
    A = np.maximum(0, Z)  # compute the ReLU
    assert(A.shape == Z.shape) # check the shape of ReLU

    cache = Z  # store the ReLU in cache

    return A, cache


In [6]:
def L_model_forward(X, parameters): # compute the forward propagation

    caches = [] # initialize a list for storing cache
    A = X # initialize the activation A
    L = len(parameters) // 2 # number of layers in the network

    # compute the activation A for each layer
    for l in range(1, L):
        A_prev = A # store the previous activation A
        A, cache = activation(A_prev, parameters['w'+str(l)], parameters['b'+str(l)], "relu") # compute the activation A for the current layer
        caches.append(cache) # store the cache in the list

    AL, cache = activation(A, parameters['w'+str(L)], parameters['b'+str(L)], "relu") # compute the activation A for the last layer
    caches.append(cache) # store the cache in the list

    assert(AL.shape == (1, X.shape[1])) # check the shape of activation A

    return AL, caches

In [7]:
# compute the cost function

def cost_function(AL, Y):

    # Mean squared error cost function
    cost = np.sum(np.square(AL - Y)) / Y.shape[1]
    assert(cost.shape == ()) # check the shape of cost
    
    return cost

In [8]:
# update the parameters using gradient descent

def update_params(parameters, grads, learning_rate):

    L = len(parameters) // 2 # number of layers in the network

    for l in range(L):
        parameters['w'+str(l+1)] = parameters['w'+str(l+1)] - learning_rate * grads['dw'+str(l+1)] # update the weights
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] - learning_rate * grads['db'+str(l+1)] # update the bias

    return parameters

In [9]:
def linear_backward(dZ, cache):
    # retrieve values from the first cache
    A_prev, W, b = cache

    m = A_prev.shape[1]

    dW = np.dot(dZ, A_prev.T) / m # compute the gradient of the cost with respect to the weights
    db = np.sum(dZ, axis=1, keepdims=True) / m # compute the gradient of the cost with respect to the bias
    dA_prev = np.dot(W.T, dZ) # compute the gradient of the cost with respect to the activation A_prev

    assert (dA_prev.shape == A_prev.shape) 
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)

    return dA_prev, dW, db

In [10]:
def sigmoid_backward(dA, activation_cache):
    Z = activation_cache

    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)

    assert (dZ.shape == Z.shape)

    return dZ

def relu_backward(dA, activation_cache):
    Z = activation_cache

    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    dZ[Z <= 0] = 0

    assert (dZ.shape == Z.shape)

    return dZ

In [12]:
# linear activation backward propagation

def linear_activation_backward(dA, cache, activation_type):

    linear_cache, activation_cache = cache # retrieve the cache

    if activation_type == "relu":
        dZ = relu_backward(dA, activation_cache) # compute the gradient of the cost with respect to the activation A_prev
        dA_prev, dW, db = linear_backward(dZ, linear_cache) # compute the gradient of the cost with respect to the activation A_prev

    elif activation_type == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache) # compute the gradient of the cost with respect to the activation A_prev
        dA_prev, dW, db = linear_backward(dZ, linear_cache) # compute the gradient of the cost with respect to the activation A_prev

    else:
        print("Wrong activation type!")
        print("Please choose from sigmoid or relu!")
        exit(1)

    return dA_prev, dW, db

In [13]:
# backward propagation

def backward_pg(Al, Y, caches):

    grads = {} # initialize a dictionary for storing gradients
    L = len(caches) # number of layers in the network
    Y = Y.reshape(AL.shape) # reshape Y

    dAL = - (np.divide(Y, Al) - np.divide(1 - Y, 1 - Al)) # compute the derivative of cost function with respect to activation A

    current_cache = caches[L-1] # get the cache of the last layer
    grads["dA"+str(L)], grads["dw"+str(L)], grads["db"+str(L)] = linear_activation_backward(dAL, current_cache, "relu") # compute the gradients of the last layer

    for l in reversed(range(L-1)): # iterate over the layers in reverse order
        current_cache = caches[l] # get the cache of the current layer
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+2)], current_cache, "relu") # compute the gradients of the current layer
        grads["dA"+str(l+1)] = dA_prev_temp # store the gradients of the current layer
        grads["dw"+str(l+1)] = dW_temp # store the gradients of the current layer
        grads["db"+str(l+1)] = db_temp # store the gradients of the current layer

    return grads

In [15]:
# train the model

layer_dims = [8, 16, 1] # the dimensions of the network

def train_model(X, Y, layer_dims, learning_rate = 0.001, num_iter = 500, print_cost = False): # train the model

    np.random.seed(1) # initialize the seed for generating random numbers
    cost = [] # initialize a list for storing cost

    parameters = initialize_parameters(layer_dims) # initialize the parameters

    for i in range(num_iter): # iterate for num_iter times
        
        AL, caches = L_model_forward(X, parameters) # compute the forward propagation
        cost = cost_function(AL, Y) # compute the cost function
        grads = backward_pg(AL, Y, caches) # compute the backward propagation
        parameters = update_params(parameters, grads, learning_rate) # update the parameters

        if print_cost and i % 100 == 0: # print the cost every 100 iterations
            print("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0: # append the
            cost.append(cost)
        
        plt.plot(np.squeeze(cost)) # plot the cost
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

    return parameters




# train the model


#Training accuracy

# test accuracy